### Setting up a basic training loop - not using attention

In [1]:
''' Changing directories '''
import os 
if 'BERT' not in os.getcwd():
    os.chdir('BERT')
print("Current working dir is {}".format(os.getcwd()))

Current working dir is /juice/scr/scr110/scr/nlp/mtl_bert/unidirectional-NMT/BERT


In [3]:
import torch
import onmt
from dataset import TextDataset
from encoder import Encoder 
from lib.huggingface.transformers import RobertaTokenizer, CamembertTokenizer
from torch.utils.data import DataLoader

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
euro_parl_dataset = 'data/data-30k-default/' # specifying the dataset that we want to use

In [6]:
text_dataset = TextDataset(euro_parl_dataset, is_train=True)

Loading data from file: data.train.2.pt
Loading data from file: data.train.1.pt
Loading data from file: data.train.0.pt
removed 11332 examples - not long enough


In [10]:
english_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
french_tokenizer = CamembertTokenizer.from_pretrained('camembert-base')

In [9]:
def collate(data): 
    '''Collating function to be passed into the dataloader '''
    input_sentences, output_sentences = zip(*data)
    input_lengths = [len(sentence)+2 for sentence in input_sentences]
    output_lengths = [len(sentence)+2 for sentence in output_sentences]
    
    batch_size = len(input_sentences)
    
    max_input_lengths = max(input_lengths)
    max_output_lengths = max(output_lengths)
    
    input_idx_tensor = torch.zeros((batch_size, max_input_lengths), dtype=torch.long)
    output_idx_tensor = torch.zeros((batch_size, max_output_lengths), dtype=torch.long)
    
    for idx, (sentence_len, input_sentence) in enumerate(zip(input_lengths, input_sentences)): 
        input_idx_tensor[idx, :] = torch.tensor(english_tokenizer.encode(input_sentence) + [1]*(max_input_lengths-sentence_len))

    
    for idx, (sentence_len, output_sentence) in enumerate(zip(output_lengths, output_sentences)): 
        output_idx_tensor[idx, :] = torch.tensor(french_tokenizer.encode(output_sentence) + [1]*(max_output_lengths-sentence_len))

    return (input_idx_tensor, output_idx_tensor)

In [11]:
dataloader = DataLoader(text_dataset, batch_size=64,
                        shuffle=True, num_workers=4, collate_fn=collate)

In [12]:
try:
    del english_encoder
except:
    pass 

#### Specifying the encoding and decoding models

In [22]:
english_encoder = Encoder("english")
french_encoder = Encoder("french")

100%|██████████| 445032417/445032417 [00:27<00:00, 16160303.16B/s]


In [30]:
print(english_encoder._modules['model'].embeddings)

RobertaEmbeddings(
  (word_embeddings): Embedding(50265, 768, padding_idx=1)
  (position_embeddings): Embedding(514, 768, padding_idx=1)
  (token_type_embeddings): Embedding(1, 768)
  (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)


In [ ]:
embeddings = # 
small_transfomer_config = {"num_layers" : 6,
"d_model" : 512,
"heads": 8,
"d_ff":2048,
"copy_attn":False,
"self_attn_type":"scaled-dot",
"dropout":0.1,
"attention_dropout":0.1,
"embeddings": #english_encoder._modules['model'].embeddings, (?) 
"max_relative_positions":,
"aan_useffn":,
}

In [21]:
french_decoder = 
english_decoder = onmt.decoder.TransformerDecoder(**small_transfomer_config)  

NameError: name 'num_layers' is not defined

In [ ]:
def loss_fn(encoder_output_english, encoder_output_french, decoder_output_french, decoder_output_english):
    '''Adversarial Loss'''
    return None 

In [18]:
def train(encoder_model, decoder_model, data_iter): 
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    
    for batch in data_iter:
        optimizer.zero_grad()
        english_sentences, french_sentences = batch
        
        # using the pooled output 
        _, encoder_output_english = encoder_model(english_sentences)
        #_, encoder_output_french = encoder_model(french_sentences)
        
        
        #loss = loss_fn(output, target)
        #loss.backward()
        #optimizer.step()

        return

In [19]:
train(english_encoder, None, dataloader)

torch.Size([64, 48])
2
torch.Size([64, 48, 768])
torch.Size([64, 768])


In [20]:
english_encoder

Encoder(
  (model): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-05, element

(tensor([    0,   133, 25616, 36907,   354,   627,  7415,  1409, 16084,     3,
          261,     3,  1116,   627,     3,   261,   627,     3, 22649, 13716,
          463, 29337,     3,   261,   627, 36242,     3,   560,     3,     3,
            3,  7761,     3, 28986,   463,  6991, 33798,     3,     2]), tensor([    0,     3, 10350,  2300,     3,   990,   459,     3,  2794, 16084,
            3,  1180, 13681,  2794,  2560,     3,  2794,     3,  2794,  2560,
            3,     3,   594,  2794,  2560,     3,     3, 12557,  2560,     3,
            3,  2794,     3, 10067,     3,  2794,     3, 10067,     3,   594,
            3,     3,     3,     2]))
(tensor([    0,   133, 44503,   354, 32632,   560, 14035, 35369,     3,  1409,
            3,  1116,  2629,     3,     3,     2]), tensor([    0, 10766, 44503,     3,  2794,  1634,     3,  2794,     3,     3,
         5489,   459,     3,  2794, 11146,     3,     3,     2]))
(tensor([    0, 20697,     3,  1409, 10980,     3,   261,     3,  1

In [9]:
print(_src_vocab)

['<unk>', '<blank>', 'the', 'of', 'to', 'and', 'in', 'is', 'a', 'that', 'I', 'for', 'on', 'be', 'this', 'we', 'are', 'have', 'not', 'it', 'as', 'The', 'with', 'European', 'which', 'will', 'by', 'has', 'Mr', 'an', 'would', 'We', 'at', 'also', 'should', 'must', 'our', 'from', 'been', 'all', 'It', 'but', 'Commission', 'President,', '-', 'can', 'was', 'you', 'Member', 'more', 'its', 'This', 'like', 'very', 'their', 'do', 'In', 'there', 'or', 'these', 'they', 'about', 'what', 'one', 'need', 'Parliament', 'report', 'my', 'Union', 'only', 'other', 'important', 'Council', 'new', 'EU', 'who', 'no', 'States', 'us', 'am', 'if', 'so', 'because', 'take', 'make', 'into', 'up', 'such', 'support', 'people', 'between', 'Europe', 'now', 'countries', 'those', 'some', 'out', 'any', 'being', 'when', 'were', 'than', 'political', 'therefore', 'That', 'made', 'believe', 'many', 'policy', 'time', 'economic', 'work', 'against', 'first', 'think', 'just', 'cannot', 'social', 'As', 'say', 'way', 'does', 'There', '

In [81]:
len(vocab['src'].fields[0][1].vocab.itos)

30002

In [82]:
len(vocab['tgt'].fields[0][1].vocab.itos)

30004

In [ ]:
import os 
import torch

In [2]:
if 'BERT' not in os.getcwd():
    os.chdir('BERT')
print("Current working dir is {}".format(os.getcwd()))

Current working dir is /juice/scr/scr110/scr/nlp/mtl_bert/unidirectional-NMT/BERT


In [3]:
data_root = os.getcwd() + '/data/data-30k-default/'

In [4]:
vocab = torch.load(data_root + 'data.vocab.pt')

In [5]:
from lib.inputter import old_style_vocab

In [6]:
using_old_style_vocab = old_style_vocab(vocab)
print(using_old_style_vocab)

False


In [7]:
fields = vocab

In [8]:
for side in ['src', 'tgt']:
        f = fields[side]
        try:
            f_iter = iter(f)
        except TypeError:
            f_iter = [(side, f)]
        for sn, sf in f_iter:
            if sf.use_vocab:
                print(' * %s vocab size = %d' % (sn, len(sf.vocab)))

 * src vocab size = 30002
 * tgt vocab size = 30004


In [9]:
data_path = data_root + "data"

In [10]:
print(data_path)

/juice/scr/scr110/scr/nlp/mtl_bert/unidirectional-NMT/BERT/data/data-30k-default/data


In [11]:
import glob 
from lib.inputter import build_dataset_iter

In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
train_iter = build_dataset_iter("train", data_path, fields)

In [13]:
valid_iter = build_dataset_iter("valid", data_path, fields, is_train=False)

In [ ]:
def test_iter():
    for iter_idx, batch in enumerate(train_iter):
        if (iter_idx == 4):
            return
        print(batch)
        print(batch.tgt[1:, :, 0])

test_iter()

This file is just a basic demo I created in order to understand how the ONMT code is structured !

Testing out how the pytorch TextDataset is structured and experiment with creating a basic neural network that uses the textDataset as input

In [2]:
import os 
import torch

Current working dir is /juice/scr/scr110/scr/nlp/mtl_bert/unidirectional-NMT/BERT


In [15]:
def load_data(data_folder, is_train):
    root_directory_path = os.path.join(os.getcwd(), data_folder)
    for filename in os.listdir(root_directory_path):
        if(is_train):
            if('.train' not in filename):
                 continue

        _torch_dataset = torch.load(os.path.join(root_directory_path,filename))
        print(_torch_dataset[0].__dict__.keys())
        print(_torch_dataset[0].src)
        return
        print("Found file with name {filename}".format(filename=filename))


In [18]:
import torchtext

# how do we specify what the initial values of the embeddings should be? 

In [19]:
class OrderedIterator(torchtext.data.Iterator):

    def __init__(self,
                 dataset,
                 batch_size,
                 pool_factor=1,
                 batch_size_multiple=1,
                 yield_raw_example=False,
                 **kwargs):
        super(OrderedIterator, self).__init__(dataset, batch_size, **kwargs)
        self.batch_size_multiple = batch_size_multiple
        self.yield_raw_example = yield_raw_example
        self.dataset = dataset
        self.pool_factor = pool_factor

    def create_batches(self):
        if self.train:
            if self.yield_raw_example:
                self.batches = batch_iter(
                    self.data(),
                    1,
                    batch_size_fn=None,
                    batch_size_multiple=1)
            else:
                self.batches = _pool(
                    self.data(),
                    self.batch_size,
                    self.batch_size_fn,
                    self.batch_size_multiple,
                    self.sort_key,
                    self.random_shuffler,
                    self.pool_factor)
        else:
            self.batches = []
            for b in batch_iter(
                    self.data(),
                    self.batch_size,
                    batch_size_fn=self.batch_size_fn,
                    batch_size_multiple=self.batch_size_multiple):
                self.batches.append(sorted(b, key=self.sort_key))

    def __iter__(self):
        """
        Extended version of the definition in torchtext.data.Iterator.
        Added yield_raw_example behaviour to yield a torchtext.data.Example
        instead of a torchtext.data.Batch object.
        """
        while True:
            self.init_epoch()
            for idx, minibatch in enumerate(self.batches):
                # fast-forward if loaded from state
                if self._iterations_this_epoch > idx:
                    continue
                self.iterations += 1
                self._iterations_this_epoch += 1
                if self.sort_within_batch:
                    # NOTE: `rnn.pack_padded_sequence` requires that a
                    # minibatch be sorted by decreasing order, which
                    #  requires reversing relative to typical sort keys
                    if self.sort:
                        minibatch.reverse()
                    else:
                        minibatch.sort(key=self.sort_key, reverse=True)
                if self.yield_raw_example:
                    yield minibatch[0]
                else:
                    yield torchtext.data.Batch(
                        minibatch,
                        self.dataset,
                        self.device)
            if not self.repeat:
                return



In [ ]:
''' TODO: for a given dataset try to remove the entries that are empty'''
def remove_empty(torch_dataset): 
    for example in torch_dataset: 
        pass

In [16]:
europarl_data_path = 'data/data-30k-default'
load_data(europarl_data_path, is_train=True)

dict_keys(['indices', 'src', 'tgt'])
[['Paradoxically,', 'the', 'European', 'Union', 'has', 'not', 'yet', 'introduced', 'in', 'all', 'its', 'Member', 'States', 'as', 'undeniably', 'important', 'a', 'topic', 'as', 'oncology.']]
